In [1]:
import pandas as pd

df = pd.read_excel('pd练习：015：根据区间（开闭条件）得出结果.xlsx', usecols='A:E')
df

,指标名称,低风险阈值,中风险阈值,高风险阈值,指标值
0,指标A,"(70%,100%]","[10%,70%]","[0,10%)",0.80
1,指标B,"[90%,110%]","[80%,90%)或(110%,120%]","[0,80%)或(120%,+∞]",0.75
2,指标C,100%,"(85%,100%)","[0,85%]",1.00


In [88]:
df.iloc[:, 1:].apply(
    lambda x: x.iloc[:-1]
    .str.replace(
        r'(?:\(|\[)(\d+)%?,(\d+|\+∞)%?(?:\)|\])',
        rf'((\1<={(n:=x.iloc[-1]*100)})&({n}<\2))',
        regex=True,
    )
    .str.replace(r'或', '|')
    .str.replace('+∞', '9999')
    .str.replace(r'(\d+)%', rf'{n}==\1', regex=True)
    .map(eval)
    .idxmax(),
    axis=1,
)

0    低风险阈值
1    高风险阈值
2    低风险阈值
dtype: object

In [ ]:
df2 = (
    df.iloc[:, :-1]
    .melt('指标名称', var_name='风险等级')
    .assign(
        value=lambda x: x.value.str.replace('+∞', r'9999%').str.split('或'),
        风险等级=lambda x: x.风险等级.str[0],
    )
    .explode('value')
)
df2[[0, 1]] = df2.value.str.extract(r'(\d+)%?,?(\d+)?')
df2[1] = df2[1].fillna(df2[0])
df2[[0, 1]] = df2[[0, 1]].astype(float) / 100
df2.drop('value', axis=1, inplace=True)
g = df2.groupby('指标名称')

def get_g(S: pd.Series) -> str:
    df = g.get_group(S.指标名称)
    a = df[0] <= S.指标值
    b = df[1] > S.指标值
    c = (df[[0, 1]] == S.指标值).all(axis=1)
    r = ';'.join(df.loc[(a & b) | c, '风险等级'])
    return r

df['风险等级'] = df.apply(get_g, axis=1)
df

,指标名称,低风险阈值,中风险阈值,高风险阈值,指标值,风险等级
0,指标A,"(70%,100%]","[10%,70%]","[0,10%)",0.80,低
1,指标B,"[90%,110%]","[80%,90%)或(110%,120%]","[0,80%)或(120%,+∞]",0.75,高
2,指标C,100%,"(85%,100%)","[0,85%]",1.00,低
